In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy

In [2]:
from isaac.utils import get_cuda_device_if_available, create_directory
device = get_cuda_device_if_available()
print(device)

data_directory = "xy_vxvy/"
create_directory(data_directory)
create_directory("models/xy_vxvy/")
create_directory("scalers")

cuda:0


In [3]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.utils import plot_confusion_matrix
from isaac.constants import BASIC_TRAINING_COLS, FORCE_CLASS_COLS, MASS_CLASS_COLS
from isaac.training import training_loop
from isaac.models import MultiBranchModel, initialise_model
from isaac.evaluation import get_best_model_and_its_accuracy

In [4]:
BATCH_SIZE = 128
EPOCHS = 60
NORMALISE_DATA = True
STEP_SIZE = 3
SEQ_END = 2700

INPUT_DIM = len(BASIC_TRAINING_COLS)   # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

network_params = (INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

## Read dataset and preprocess it

In [5]:
train_trials = read_dataset("data/r_train_trials.h5")
val_trials = read_dataset("data/r_val_trials.h5")

train_classes = [trial.combined_solution.iloc[0] for trial in train_trials]
val_classes = [trial.combined_solution.iloc[0] for trial in val_trials]

np.random.seed(37)
train_trials, _ = train_test_split(train_trials, train_size=4000, stratify=train_classes)
val_trials, _ = train_test_split(val_trials, train_size=1000, stratify=val_classes)

100%|██████████| 2735/2735 [00:30<00:00, 90.97it/s]


## Train model and plot loss and accuracy

In [6]:
N_MODELS = 25

stats_dfs = []
loaders, scaler = prepare_dataset([train_trials, val_trials], 
                                  class_columns=[list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)], 
                                  multiclass=True,
                                  batch_size=BATCH_SIZE, normalise_data=NORMALISE_DATA,
                                  device=device, training_columns=BASIC_TRAINING_COLS)


for seed in range(N_MODELS):
    df = pd.DataFrame()

    model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device, arch=MultiBranchModel)
    epoch_losses, epoch_accuracies, [best_mass_model, best_force_model] = training_loop(model, optimizer, 
                                                                                        error,
                                                                                        loaders[0], loaders[1], 
                                                                                        EPOCHS, seq_end=SEQ_END,
                                                                                        step_size=STEP_SIZE,
                                                                                        multibranch=True)

    torch.save(best_mass_model.state_dict(), "models/xy_vxvy/best_mass_model_seed_%d.pt" % seed)
    torch.save(best_force_model.state_dict(), "models/xy_vxvy/best_force_model_seed_%d.pt" % seed)
    
    
    train_accuracies = np.array(epoch_accuracies[0])
    val_accuracies = np.array(epoch_accuracies[1]) 
    
    df["Epoch"] = np.arange(EPOCHS)
    df["Mass Loss"] = epoch_losses[:, 0]
    df["Force Loss"] = epoch_losses[:, 1]
    df["Mass Train Accuracy"] = train_accuracies[:, 0]
    df["Mass Val Accuracy"] = val_accuracies[:, 0]
    df["Force Train Accuracy"] = train_accuracies[:, 1]
    df["Force Val Accuracy"] = val_accuracies[:,1]
    df["seed"] = str(seed)
    stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf(data_directory+"stats.h5", key="stats")

100%|██████████| 1000/1000 [00:01<00:00, 766.27it/s]
Train_loss: ([0.50908969 0.40775177]) Train_acc: ([75.175 77.   ]) Val_acc: ([67.4 67.7]): 100%|██████████| 60/60 [22:36<00:00, 22.64s/it]
Train_loss: ([0.74005193 0.79424416]) Train_acc: ([61.8  47.45]) Val_acc: ([59.1 45. ]): 100%|██████████| 60/60 [22:37<00:00, 22.62s/it]  
Train_loss: ([0.85105061 0.88699806]) Train_acc: ([58.125 54.3  ]) Val_acc: ([55.6 50.1]): 100%|██████████| 60/60 [22:39<00:00, 22.63s/it]
Train_loss: ([0.80121654 0.76192341]) Train_acc: ([61.15  58.575]) Val_acc: ([58.7 54.8]): 100%|██████████| 60/60 [22:37<00:00, 22.61s/it]
Train_loss: ([0.6551912  0.60006597]) Train_acc: ([67.275 49.85 ]) Val_acc: ([65.3 48.6]): 100%|██████████| 60/60 [22:41<00:00, 22.74s/it]
Train_loss: ([0.69241616 0.65817352]) Train_acc: ([64.775 62.15 ]) Val_acc: ([58.6 54.3]): 100%|██████████| 60/60 [22:45<00:00, 22.75s/it]
Train_loss: ([0.48770538 0.50130782]) Train_acc: ([74.75 78.  ]) Val_acc: ([67.1 67.7]): 100%|██████████| 60/60 [

## Save model and scaler

In [7]:
joblib.dump(scaler, "scalers/passive_xy_vxvy_scaler.sk")

['scalers/passive_xy_vxvy_scaler.sk']